# Based on:
- https://www.kaggle.com/code/mgmostafa/cmi-best-single-model-simplified/output
- https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet

# Notes on Successful approaches:
- The best performing public model uses Ensemble learning, specifically a **Voting Regressor**, which combines predictions from LightGBM, XGBoost and CatBoost. "This approach is beneficial as it leverages the strengths of multiple models, reducing overfitting and improving overall model performance."

In [328]:
import numpy as np
import pandas as pd
import os
import re
from typing import List

from scipy import stats

from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
# from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
# from sklearn.impute import SimpleImputer, KNNImputer
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, cohen_kappa_score
from scipy.stats import randint, uniform
from sklearn.feature_selection import mutual_info_classif
from scipy.optimize import minimize

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import matplotlib.cm as cm
import seaborn as sns

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [329]:
def quadratic_weighted_kappa(y_true, y_pred):
    y_pred_rounded = np.round(y_pred).clip(0, 3).astype(int)
    return cohen_kappa_score(y_true, y_pred_rounded, weights='quadratic')

In [330]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    
    # Calculate time gaps
    df['time'] = pd.to_datetime(df['time_of_day'])
    df['time_gap'] = df['time'].diff().dt.total_seconds()
    
    # Define activity/inactivity
    enmo_threshold = 0.003
    non_wear = df['non-wear_flag'] == 1
    low_enmo = df['enmo'] < enmo_threshold
    large_gap = df['time_gap'] > 5

    is_inactive = non_wear | low_enmo | large_gap
    # print(f"Total inactive records: {is_inactive.sum()} ({is_inactive.mean()*100:.2f}%)")
    # print(f"Total active records: {(~is_inactive).sum()} ({(~is_inactive).mean()*100:.2f}%)")

    # print(f"\nDebugging file {filename}:")
    # print(f"Total records: {len(df)}")
    # print(f"Records with non-wear flag: {non_wear.sum()} ({non_wear.mean()*100:.2f}%)")
    # print(f"Records with low ENMO: {low_enmo.sum()} ({low_enmo.mean()*100:.2f}%)")
    # print(f"Records with large gaps: {large_gap.sum()} ({large_gap.mean()*100:.2f}%)")

    active_df = df[~is_inactive].copy()  # Get only active periods
    
    df['hour'] = df['time'].dt.hour
    active_df['hour'] = active_df['time'].dt.hour
    
    # Basic stats for all periods
    all_basic_stats = df[['enmo', 'anglez', 'X', 'Y', 'Z', 'light', 'battery_voltage', 'time_gap']].describe().values.reshape(-1)
    
    # Basic stats for active periods only
    active_basic_stats = active_df[['enmo', 'anglez', 'X', 'Y', 'Z', 'light', 'battery_voltage', 'time_gap']].describe().values.reshape(-1)


    daytime_enmo = df[df['hour'].between(9, 17)]['enmo'].mean() or 0   # removed comma
    nighttime_enmo = df[~df['hour'].between(9, 17)]['enmo'].mean() or 0
    active_daytime = active_df[active_df['hour'].between(9, 17)]['enmo'].mean() or 0   # removed comma
    active_nighttime = active_df[~active_df['hour'].between(9, 17)]['enmo'].mean() or 0

    
    # Additional features for all periods
    all_key_features = [
        (~is_inactive).mean(),  # Activity ratio
        daytime_enmo,
        nighttime_enmo,
        # df[df['hour'].between(9, 17)]['enmo'].mean(),  # Daytime activity
        # df[~df['hour'].between(9, 17)]['enmo'].mean(),  # Non-daytime activity
        df['time_gap'].gt(5).mean()  # Proportion of large gaps
    ]
    
    # Additional features for active periods
    active_key_features = [
        active_daytime,
        active_nighttime,
        # active_df[active_df['hour'].between(9, 17)]['enmo'].mean(),  # Active daytime activity
        # active_df[~active_df['hour'].between(9, 17)]['enmo'].mean(),  # Active non-daytime activity
        active_df['time_gap'].gt(5).mean()  # Proportion of large gaps in active periods
    ]
    
    # print("\nFeature stats:")
    # print("All basic stats shape:", all_basic_stats.shape)
    # print("Active basic stats shape:", active_basic_stats.shape)
    # print("All key features:", all_key_features)
    # print("Active key features:", active_key_features)
    
    features = np.concatenate([all_basic_stats, all_key_features, active_basic_stats, active_key_features])
    # print("Final features shape:", features.shape)
    # print("Any NaN in features:", np.isnan(features).any())
    
    return features, filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)

    # test_ids = ids[:5] # temp
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(
            executor.map(lambda fname: process_file(fname, dirname), ids),
            total=len(ids))
            # executor.map(lambda fname: process_file(fname, dirname), test_ids),
            # total=len(test_ids))
            
        )
    
    stats, indexes = zip(*results)
    
    # Column names
    cols = []
    # For all periods
    for var in ['enmo', 'anglez', 'X', 'Y', 'Z', 'light', 'battery_voltage', 'time_gap']:
        for stat in ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']:
            cols.append(f'all_{var}_{stat}')
    
    cols.extend(['activity_ratio', 'all_daytime_activity', 'all_nighttime_activity', 'all_gap_ratio'])
    
    # For active periods
    for var in ['enmo', 'anglez', 'X', 'Y', 'Z', 'light', 'battery_voltage', 'time_gap']:
        for stat in ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']:
            cols.append(f'active_{var}_{stat}')
    
    cols.extend(['active_daytime_activity', 'active_nighttime_activity', 'active_gap_ratio'])
    
    df = pd.DataFrame(stats, columns=cols)
    df['id'] = indexes
    
    return df

In [331]:
# # MOST BASIC PROCESSING OF PARQUET (JUST GETS SUMMARY STATS FOR EACH PARQUET FEATURE)

# def process_file(filename, dirname):
#     df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
#     df.drop('step', axis=1, inplace=True)
#     return df.describe().values.reshape(-1), filename.split('=')[1]

# def load_time_series(dirname) -> pd.DataFrame:
#     ids = os.listdir(dirname)
    
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(
#             executor.map(lambda fname: process_file(fname, dirname), ids),
#             total=len(ids))
#         )
    
#     stats, indexes = zip(*results)
    
#     # Create generic column names for all statistics
#     df = pd.DataFrame(stats, columns=[f"Stat_{i}" for i in range(len(stats[0]))])
#     df['id'] = indexes
    
#     return df

In [332]:
def preprocess_data(df: pd.DataFrame, is_train: bool = False, encoders: dict = None, return_encoders: bool = False) -> pd.DataFrame:
    """Preprocess data including encoding and imputation"""
    df = df.copy()
    
    # First handle missing values
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numerical_cols:
        df[col] = df[col].fillna(df[col].median())
        
    # Identify categorical columns (including seasons)
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    # Handle missing categoricals
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    # Encode categorical variables
    if is_train:
        # For training data, create new encoders
        encoders = {}
        for col in categorical_cols:
            if col != 'id':  # Skip ID column
                encoders[col] = LabelEncoder()
                df[col] = encoders[col].fit_transform(df[col])
        # Decide whether to return encoders
        return (df, encoders) if return_encoders else df
    else:
        if encoders is None:
            raise ValueError("Must provide encoders when preprocessing test data")
        # For test data, use provided encoders
        for col in categorical_cols:
            if col != 'id' and col in encoders:  # Skip ID and handle only columns present in train
                df[col] = encoders[col].transform(df[col])
        return df

In [333]:
def process_features(main_df: pd.DataFrame, parquet_features: pd.DataFrame) -> pd.DataFrame:
    """Process all features in one go"""
    # If supervised_df is a tuple (from preprocess_data), take just the DataFrame
    if isinstance(main_df, tuple):
        main_df = main_df[0] # don't need encoders object
    
    # OPTIONAL: GENERATE MORE FEATURES (Removed for now)
    # feature_generator = ActivityFeatureGenerator()
    # df_features = feature_generator.generate_features(parquet_features)
    
    # Join with original data
    df_combined = pd.merge(
        main_df,
        # df_features,
        parquet_features,
        on='id',
        how='left'
    )
    
    return df_combined

In [334]:
def find_correlated_features(X, threshold=0.95):
    """Identifies correlated features to remove based on training data"""
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features to drop
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(f"Identified {len(to_drop)} correlated features to remove")
    return to_drop

def remove_correlated_features(X, features_to_drop):
    """Removes specified features from a dataset"""
    X_reduced = X.drop(columns=features_to_drop)
    return X_reduced

# Make a quick model

In [335]:
class_weights = {
    0: 1.0,
    1: 2.2,  
    2: 4.2,  
    3: 48.0  
}

In [336]:
%%time
# Functions for training the evaluating the selected model 
def threshold_Rounder(oof_non_rounded, thresholds):
    buffer2 = 0.2  # Buffer for class 2
    buffer3 = 0.3  # Larger buffer for class 3
    return np.where(oof_non_rounded < thresholds[0], 0,
                   np.where(oof_non_rounded < thresholds[1], 1,
                           np.where(oof_non_rounded < thresholds[2] - buffer2, 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)


CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 17.4 µs


In [337]:
import torch

def get_device_params():
    # if os.environ.get('KAGGLE_IS_KAGGLE_GPU_ENABLED', '') == 'True':
    if torch.cuda.is_available():
        XGB_device = {'tree_method': 'gpu_hist'}
        CB_device = {'task_type': 'GPU'}
        print("GPU detected - using GPU accelerated training")
    else:
        XGB_device = {'tree_method': 'hist'}
        CB_device = {'task_type': 'CPU'}
        print("No GPU detected - using CPU training")
    return XGB_device, CB_device

In [338]:
xgb_device, cb_device = get_device_params()

SEED = 42
n_splits = 5

LGB_Params = {
    'learning_rate': 0.07, 
    'random_state': SEED, 
    'n_estimators': 200,
    
    'max_depth': 8, 
    'num_leaves': 300, 
    'min_data_in_leaf': 17,
    # 'max_depth': 6,  # Reduced from 8
    # 'min_data_in_leaf': 10,  # Reduced from 17
    # 'num_leaves': 150,  # Reduced from 300
    
    'feature_fraction': 0.7689, 
    'bagging_fraction': 0.6879, 
    'bagging_freq': 2, 
    'lambda_l1': 4.74, 
    'lambda_l2': 4.743e-06,
    'verbose': -1,
}

XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': SEED,
    **xgb_device  # This will add either GPU or CPU parameters
}


CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 10,
    **cb_device  # This will add either GPU or CPU parameters

}

GPU detected - using GPU accelerated training


# Try ensembling

In [339]:
models = {
    'lgbm': LGBMRegressor(**LGB_Params),
    'xgboost': XGBRegressor(**XGB_Params),
    'catboost': CatBoostRegressor(**CatBoost_Params)
}

In [340]:
def analyze_predictions_distribution(y_true, predictions, name=""):
    print(f"\n{'-'*20} {name} Prediction Analysis {'-'*20}")
    
    # Get distribution of predictions
    pred_dist = pd.Series(predictions).value_counts(normalize=True).sort_index() * 100
    
    if y_true is not None:
        # Get distribution of true values
        true_dist = pd.Series(y_true).value_counts(normalize=True).sort_index() * 100
        
        print("\nClass Distribution Comparison:")
        comparison_df = pd.DataFrame({
            'True %': true_dist,
            'Predicted %': pred_dist
        }).round(2)
        print(comparison_df)
        
        # Confusion matrix for detailed analysis
        conf_matrix = pd.crosstab(y_true, predictions, 
                                normalize=True) * 100  # Changed from 'true' to True
        print("\nConfusion Matrix (% of true classes predicted as each class):")
        print(conf_matrix.round(2))
        
        # Analysis of severe cases (class 3)
        if 3 in y_true:
            severe_mask = y_true == 3
            severe_predictions = predictions[severe_mask]
            print(f"\nDetailed Analysis of Severe Cases (Class 3):")
            print(f"Total Severe Cases: {sum(severe_mask)}")
            print("Predicted as:")
            for pred_class in range(4):
                count = sum(severe_predictions == pred_class)
                percent = (count / sum(severe_mask)) * 100
                print(f"Class {pred_class}: {count} cases ({percent:.2f}%)")
    else:
        print("\nPrediction Distribution:")
        for class_val, percentage in pred_dist.items():
            print(f"Class {class_val}: {percentage:.2f}%")

In [341]:
def TrainML(model_class, test_data):
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        
        # Create sample weights for all models
        sample_weights = np.array([class_weights[yi] for yi in y_train])
        
        # All models use sample_weight
        model.fit(X_train, y_train, sample_weight=sample_weights)
        
        # model = clone(model_class)
        # model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)
        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded
        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)
        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
    
    analyze_predictions_distribution(y, oof_rounded, "OOF")
    
    mean_val_score = np.mean(test_S)
    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {mean_val_score:.4f}")
    
    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)
    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': test_id,
        'sii': tpTuned
    })
    
    return submission, model, tKappa

def weighted_ensemble_train(X, y, test_data, models_dict):
    predictions = {}
    val_scores = {}
    
    for name, model in models_dict.items():
        print(f"\nTraining {name}...")
        submission, trained_model, val_score = TrainML(model, test_data)
        predictions[name] = submission['sii']
        val_scores[name] = val_score
        print(f"{name} completed with score: {val_score:.4f}")
    
    # Manual weights favoring CatBoost
    weights = {
        'lgbm': 0.25,
        'xgboost': 0.25,
        'catboost': 0.50 
    }
    
    print("\nModel Weights:")
    for name, weight in weights.items():
        print(f"{name}: {weight:.3f}")
    
    # Weighted average
    ensemble_pred = np.zeros_like(list(predictions.values())[0], dtype=float)
    for name, pred in predictions.items():
        ensemble_pred += weights[name] * pred

    final_pred = threshold_Rounder(ensemble_pred, [0.5, 1.5, 2.5])
    analyze_predictions_distribution(None, final_pred, "Ensemble Test")
        
    final_submission = pd.DataFrame({
        'id': test_id,
        'sii': ensemble_pred.round().astype(int)
    })
    
    return final_submission, val_scores, weights

# Steps for improvement:
- Address class imbalance
    - Modify your StratifiedKFold implementation to ensure the rare class (severity 3) is properly represented in each fold:
    - Add sample weights to give more importance to minority classes:
    - Modify your threshold optimization to be more sensitive to minority classes:
    - Adjust model parameters for imbalanced data:

- Add more new features (look at what other contestants are doing)
- Modify StatifiedKFold to use group-based splitting 
- Consider making model weights adaptive based on validation performance
- Make threshold rounder more robust
- Correlation threshold of 0.95 might be too high

In [342]:
# Load data
print("Loading data...")
base_path = '/kaggle/input/child-mind-institute-problematic-internet-use'
train_parquet_path = f'{base_path}/series_train.parquet'
test_parquet_path = f'{base_path}/series_test.parquet'

# Load data
train = pd.read_csv(f'{base_path}/train.csv')
test = pd.read_csv(f'{base_path}/test.csv')

# Get supervised data (rows with non-null target)
train = train[train['sii'].notna()].copy()
print(f"Train shape: {train.shape}, Test shape: {test.shape}")

Loading data...
Train shape: (2736, 82), Test shape: (20, 59)


In [343]:
# # !ls
# print(os.getcwd())

In [344]:
# # Load parquet features with error handling
# print("\nProcessing parquet files...")
# try:
#     print("Loading and processing training data...")
#     train_ts = load_time_series(train_parquet_path)
#     print("\nShape of training features:", train_ts.shape)
    
#     print("\nLoading and processing test data...")
#     test_ts = load_time_series(test_parquet_path)
#     print("\nShape of test features:", test_ts.shape)
    
#     # Save features
#     print("\nSaving features...")
#     train_ts.to_parquet("actigraphy_features_train.parquet")
#     test_ts.to_parquet("actigraphy_features_test.parquet")
# except Exception as e:
#     print(f"Error processing parquet files: {str(e)}")
#     raise

In [345]:
# Merge with original data
print("\nMerging parquet data with CSV data...")
try:
    train_ts = pd.read_parquet("actigraphy_features_train.parquet")
    test_ts = pd.read_parquet("actigraphy_features_test.parquet")
    
    train_combined = process_features(train, train_ts)
    test_combined = process_features(test, test_ts)
    print("\nInitial shapes:")
    print("Train combined:", train_combined.shape)
    print("Test combined:", test_combined.shape)
except Exception as e:
    print(f"Error in merging: {str(e)}")
    raise


Merging parquet data with CSV data...

Initial shapes:
Train combined: (2736, 217)
Test combined: (20, 194)


In [346]:
# Preprocess with error handling
print("\nPreprocessing data...")
try:
    train_combined, encoders = preprocess_data(train_combined, is_train=True, return_encoders=True)
    test_combined = preprocess_data(test_combined, is_train=False, encoders=encoders)
except Exception as e:
    print(f"Error in preprocessing: {str(e)}")
    raise


Preprocessing data...


In [347]:
print("\nLast Prep for Training...")
try:
    exclude_patterns = ['id', 'PCIAT', 'sii']
    feature_cols = [col for col in train_combined.columns 
                   if not any(pattern in col for pattern in exclude_patterns)]
    X = train_combined[feature_cols]
    y = train_combined['sii']
    features_to_drop = find_correlated_features(X, threshold=0.95) 
    X = remove_correlated_features(X, features_to_drop)
    
    test_id = test_combined['id'].copy()
    test_features = test_combined[feature_cols]
    test_features = remove_correlated_features(test_features, features_to_drop)
except Exception as e:
    print()
    raise


Last Prep for Training...
Identified 50 correlated features to remove


In [348]:
# Train models with memory efficient processing
print("\nTraining models...")
try:
    models = {
        'lgbm': LGBMRegressor(**LGB_Params),
        'xgboost': XGBRegressor(**XGB_Params),
        'catboost': CatBoostRegressor(**CatBoost_Params)
    }
    ensemble_submission, scores, weights = weighted_ensemble_train(X, y, test_features, models)
except Exception as e:
    print(f"Error in model training: {str(e)}")
    raise

# Save submission
print("\nSaving submission...")
ensemble_submission.to_csv('submission.csv', index=False)
print("Done!")


Training models...

Training lgbm...


Training Folds:  20%|██        | 1/5 [00:00<00:02,  1.73it/s]

Fold 1 - Train QWK: 0.8556, Validation QWK: 0.3835


Training Folds:  40%|████      | 2/5 [00:01<00:01,  1.73it/s]

Fold 2 - Train QWK: 0.8488, Validation QWK: 0.3945


...

Training Folds:  80%|████████  | 4/5 [00:07<00:01,  1.85s/it]

Fold 4 - Train QWK: 0.5549, Validation QWK: 0.3389


Training Folds: 100%|██████████| 5/5 [00:09<00:00,  1.86s/it]

Fold 5 - Train QWK: 0.5642, Validation QWK: 0.3896

-------------------- OOF Prediction Analysis --------------------

Class Distribution Comparison:
     True %  Predicted %
0.0   58.26        24.56
1.0   26.68        63.27
2.0   13.82        12.10
3.0    1.24         0.07

Confusion Matrix (% of true classes predicted as each class):
col_0      0      1     2     3
sii                            
0.0    21.38  34.14  2.74  0.00
1.0     2.56  19.85  4.24  0.04
2.0     0.62   8.88  4.28  0.04
3.0     0.00   0.40  0.84  0.00

Detailed Analysis of Severe Cases (Class 3):
Total Severe Cases: 34
Predicted as:
Class 0: 0 cases (0.00%)
Class 1: 11 cases (32.35%)
Class 2: 23 cases (67.65%)
Class 3: 0 cases (0.00%)
Mean Train QWK --> 0.5461
Mean Validation QWK ---> 0.3642


----> || Optimized QWK SCORE ::  0.445
catboost completed with score: 0.4445

Model Weights:
lgbm: 0.250
xgboost: 0.250
catboost: 0.500

-------------------- Ensemble Test Prediction Analysis --------------------

Prediction Distribution:
Class 0: 20.00%
Class 1: 80.00%

Saving submission...
Done!
